In [28]:
import pandas as pd
import numpy as np
from datetime import datetime, date

import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

In [4]:
# Options
pd.set_option('display.max_columns', None)

In [5]:
# Input Timestamp
timestmp = "2023-12-02 12:15"
timestmp = datetime.strptime(timestmp, '%Y-%m-%d %H:%M')

In [21]:
# Import Features
calendar_features = pd.read_csv("raw_features_2024.csv", sep=",")
calendar_features['date'] = pd.to_datetime(calendar_features['date'], format='%Y-%m-%d') # Extract Date

In [13]:
# Get Weather-Data
## Set up Client and define Params
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://api.open-meteo.com/v1/forecast" # URL API

params = {
	"latitude": 47.4239,
	"longitude": 9.3748,
	"daily": ["temperature_2m_max", "temperature_2m_min", "rain_sum", "snowfall_sum"],
	"start_date": timestmp.strftime("%Y-%m-%d"),
	"end_date": timestmp.strftime("%Y-%m-%d")
}

In [14]:
def get_weather_forecast(params):
    
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]
    
    daily = response.Daily()
    daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
    daily_rain_sum = daily.Variables(2).ValuesAsNumpy()
    daily_snowfall_sum = daily.Variables(3).ValuesAsNumpy()

    daily_data = {"date": pd.date_range(
        start = pd.to_datetime(daily.Time(), unit = "s"),
        end = pd.to_datetime(daily.TimeEnd(), unit = "s"),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left"
    )}

    daily_data["temperature_2m_max"] = daily_temperature_2m_max
    daily_data["temperature_2m_min"] = daily_temperature_2m_min
    daily_data["rain_sum"] = daily_rain_sum
    daily_data["snowfall_sum"] = daily_snowfall_sum

    daily_dataframe = pd.DataFrame(data = daily_data)
    return(daily_dataframe)

In [19]:
df_weather = get_weather_forecast(params)

In [26]:
df = pd.merge(df_weather, calendar_features, on="date", how="left")
df["datetime"] = timestmp

In [29]:
%run ../common_code/01_prepare_time_features.ipynb
df

,date,temperature_2m_max,temperature_2m_min,rain_sum,snowfall_sum,ferien,feiertag,covid_19,olma_offa,datetime,year,month,day,weekdayname,weekday,time,hour,minute,sin_minute,cos_minute,sin_hour,cos_hour,sin_weekday,cos_weekday,sin_day,cos_day,sin_month,cos_month
0,2023-12-02,-0.8305,-6.6305,0.0,13.790002,0,0,0,0,2023-12-02 12:15:00,2023,12,2,Saturday,5,12:15,12,15,1.0,2.832769e-16,1.224647e-16,-1.0,-0.974928,-0.222521,0.406737,0.913545,-2.449294e-16,1.0
